# Python and C++ extension

## Non Linear

In [ ]:
import sys
sys.path.append('/content/CppToPython')

In [ ]:
import numpy as np
import GeDiM4Py as gedim

### Initialize

In [ ]:
lib = gedim.ImportLibrary("/content/CppToPython/release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

## Nonlinear Elliptic Problem POD vs PINNs
"Finta equazione di Burger perchè la facciamo in una sola componente"

The aim of our project is to solve the following parametrized problem on the two-dimensional spatial domain $\bar{\Omega} = [0, 1] \times [0, 1]$: given $\mathbf{\mu}=(\mu_0, \mu_1)\in\mathcal{P}=[0.1,1]\times[0.1,1]$, find $u(\mathbf{\mu})$ such that

$$
\begin{cases}
\displaystyle - \Delta u(\mathbf{\mu}) + \frac{\mu_0}{\mu_1} \left(e^{\mu_1\,u(\mathbf{\mu})}-1\right) = g(\mathbf{x};\mathbf{\mu}) & \text{in } \Omega\\
u = 0.0 & \text{in } ∂Ω
\end{cases}
$$

where the source term is defined as $g(\mathbf{x};\mathbf{\mu})=100\,sin(2\pi x_0)\,cos(2\pi x_1)\quad \forall\,\mathbf{x}=(x_0, x_1) \in\Omega$ 

$u = 16 xy(1-x)(1-y)$.

The weak form of the problem becomes, find $u \in V := H^1_0(\Omega)$
$$
\int_{\Omega} \nabla u\cdot\nabla v + \int_{\Omega} \left(e^{\mu_1\,u(\mathbf{\mu})}-1\right) v - \int_{\Omega} g\, v = 0 \quad \forall v \in V \Leftrightarrow f(u; v) := f_1(u; v) + f_2(u; v) + f_3(u; v) = 0 \quad \forall v \in V
$$

Using Newton schema, we solve for each $k$ iteration the problem
$$
J_f [\partial u]_{|_{u_k}} \partial u = - f(u_k; v) = 0 \quad \forall v \in V
$$
where $J_f [\partial u]_{|_{u_k}}$ is the evaluation of the derivative (Jacobian) of $J_f$ in the point $u_k$ along the unknown direction of $\partial u$.

Notice that the functions $f_1(u; v)$ and $f_3(u; v)$ are linear in $u$ and their Gateux derivatives are given respectively by

$$
    J_{f_1} [\partial u]_{|_{u_k}}=f_1(u; v)=\int_{\Omega} \nabla \partial u \cdot \nabla v \qquad\text{and}\qquad J_{f_3} [\partial u]_{|_{u_k}}=0
$$

On the other hand $f_3(u; v)$ is non-linear in u, and its Gateaux derivative is given by

\begin{align*}
    J_{f_2} [\partial u]_{|_{u_k}}&=\lim_{h\to0}\frac{f_2(u_k+\partial u\, h)-f(u_k)}{h}=\\
    \\
    &=\frac{\mu_0}{\mu_1}\lim_{h\to0}\frac{1}{h}\int_{\Omega} \left(e^{\mu_1\,(u_k+\partial u\, h)}-1\right) v-\int_{\Omega} \left(e^{\mu_1\,u_k}-1\right) v=\\
    \\
    &=\frac{\mu_0}{\mu_1}\lim_{h\to0}\frac{1}{h}\int_{\Omega} e^{\mu_1\,u_k}\left(e^{\mu_1\,\partial u\, h}-1\right) v=\\
    \\
    &=\mu_0\int_\Omega e^{\mu_1\,u_k}\, \partial u\,v 
\end{align*}

which is a reaction term.

After computations, we find the linear system, on each $k$ iteration, fixed $u_k$ find $\partial u$ s.t.

$$
\int_{\Omega} \nabla \partial u \cdot \nabla v + \mu_0\int_\Omega e^{\mu_1\,u_k}\, \partial u\,v  = - \int_{\Omega} \nabla u_k \cdot \nabla v - \int_{\Omega} \left(e^{\mu_1\,u_k}-1\right) v + \int_{\Omega} g\, v 
$$
In the left hand side we have the sum of the classic diffusion term and a reaction term. In the right hand side we have the residual ??

In [ ]:
#the numPoints are the quadrature points

def Diffusion(numPoints, points):
	values_d = np.ones(numPoints, order='F')
	return values_d.ctypes.data

def Reaction(numPoints, points, mu_0):
	values_r = mu_0 * np.ones(numPoints, order='F')
	return values_r.ctypes.data

def Reaction_non_linear(numPoints, points, mu_1, u):
	vecu = gedim.make_nd_array(u, numPoints, np.double)
	values_nl = np.exp(mu_1 * vecu)
	return values_nl.ctypes.data

def Burger_f(numPoints, points):
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values_f = 32.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) + matPoints[0,:] * (1.0 - matPoints[0,:])) + \
	(16.0 * (1.0 - 2.0 * matPoints[0,:]) * matPoints[1,:] * (1.0 - matPoints[1,:]) + \
	16.0 * (1.0 - 2.0 * matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:])) * \
	16.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:]))
	return values_f.ctypes.data

def Burger_non_linear_f(numPoints, points, u, u_x, u_y):
	vecu = gedim.make_nd_array(u, numPoints, np.double)
	vecu_x = gedim.make_nd_array(u_x, numPoints, np.double)
	vecu_y = gedim.make_nd_array(u_y, numPoints, np.double)
	values_nl_f = vecu * (vecu_x + vecu_y)
	return values_nl_f.ctypes.data

def Burger_non_linear_der_f(numPoints, points, u, u_x, u_y):
	vecu_x = gedim.make_nd_array(u_x, numPoints, np.double)
	vecu_y = gedim.make_nd_array(u_y, numPoints, np.double)
	values_nl_d_f = np.zeros((2, numPoints), order='F') # we have always the geometric dimension in the rows, and number of points in the columns
	values_nl_d_f[0,:] = vecu_x
	values_nl_d_f[1,:] = vecu_y
	return values_nl_d_f.ctypes.data

def Burger_exactSolution(numPoints, points): #exact solution given by the problem
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
	values_ex = 16.0 * (matPoints[1,:] * (1.0 - matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:]))
	return values_ex.ctypes.data

def Burger_exactDerivativeSolution(direction, numPoints, points): #function of the derivative in each direction
	matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)

	if direction == 0:
		values_ex_d = 16.0 * (1.0 - 2.0 * matPoints[0,:]) * matPoints[1,:] * (1.0 - matPoints[1,:])
	elif direction == 1:
		values_ex_d = 16.0 * (1.0 - 2.0 * matPoints[1,:]) * matPoints[0,:] * (1.0 - matPoints[0,:])
	else:
		values_ex_d = np.zeros(numPoints, order='F')

	return values_ex_d.ctypes.data

def Ones(numPoints, points):
	values_one = np.ones(numPoints, order='F')
	return values_one.ctypes.data

def OnesDerivative(numPoints, points):
	values_one_d = np.ones((2, numPoints), order='F')
	return values_one_d.ctypes.data

def Zeros(numPoints, points): 
	values_zero = np.zeros(numPoints, order='F')
	return values_zero.ctypes.data

def ZerosDerivative(direction, numPoints, points):  #gradient null in each direction
	values_zero_d = np.zeros(numPoints, order='F')
	return values_zero_d.ctypes.data

### Define Simulation Parameters

Set geometry parameters

In [ ]:
meshSize = 0.01 #We choose a meshsize = maximum area required in our domain discretization
order = 1 #Order of finite element

#all the boundary condition are homogenous dirichlte boundary condition so we use just one marker for the border equal to 1
domain = { 'SquareEdge': 1.0, 'VerticesBoundaryCondition': [1,1,1,1], 'EdgesBoundaryCondition': [1,1,1,1], 'DiscretizationType': 1, 'MeshCellsMaximumArea': meshSize } 
[meshInfo, mesh] = gedim.CreateDomainSquare(domain, lib)

discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 2] }
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)

Set Newton parameters

In [ ]:
residual_norm = 1.0; # variable in which we sotre the residual L2 norm
solution_norm = 1.0; # variable in which we store the solution L2 norm. They are needed in order to implement a Newton step
newton_tol = 1.0e-6 # tolerance we want to achieve with the solution approximation. It cannot be lower than the machine precision (epsilon di macchina)
max_iterations = 20 # 7 # if the mesh is not too fine it is not necessary a too high number of iteration
num_iteration = 1 

Set Initial Solution

In [ ]:
# Newton schema is a second order algorithm, ...(min.14)
u_k = np.zeros(problemData['NumberDOFs'], order='F') 
u_strong = np.zeros(problemData['NumberStrongs'], order='F')

### Run Newton Algorithm

In [ ]:
#the tolerance is relative so we multiplied it by the solution norm
#if the first is the criteria that wins there is a problem
while num_iteration < max_iterations and residual_norm > newton_tol * solution_norm: 
        #We start assembling the classic diffusion (or stiffness) matrix: we have to give it the diffusion term, the problem and the library
        #The diffusion coefficient in our case is a function that give us 1 and we call it Diffusion
    [stiffness, stiffnessStrong] = gedim.AssembleStiffnessMatrix(Diffusion, problemData, lib)

        #u_k is known at the iteration k, so it is a costant that depends on the solution k of our algorithm. 
        #We need to give to the function reaction also the solution at the previous step
        #Everything is related on something of the Newton solution at the step k is indicated by "AssembleNonLinear"
        #Again the reaction coefficient is a function called Reaction that returns every 1 multiplied by the parameter mu_0. IF WE HAVE A PARAMETER, FOR EXAMPLE \LAMBDA Burger_c IS \LAMBDA EVALUATED IN THE POINT
    [reaction, reactionStrong] = gedim.AssembleNonLinearReactionMatrix(Reaction, Reaction_non_linear, u_k, u_strong, problemData, lib)


        #We don't have the unknown, but we just have the test functions. In order to compute g we have to compute the derivative and the laplacian of the exact solution and substitute it in the equation
    forcingTerm_g = gedim.AssembleForcingTerm(Burger_f, problemData, lib)

        #We don't have any parameters, so we give again 1 as coefficient. forcingTerm_v because the sencond term of the right hand side does not depends on the derivative of the test function v, but on the function v itself
    forcingTerm_v = gedim.AssembleNonLinearForcingTerm(Ones, Burger_non_linear_f, u_k, u_strong, problemData, lib)

        #In order to compute the first term of the right hand side we have to compute the .... which is computed in Burger_non_linear_der_f. It depends on the v derivative
    forcingTerm_der_v = gedim.AssembleNonLinearDerivativeForcingTerm(OnesDerivative, Burger_non_linear_der_f, u_k, u_strong, problemData, lib)
    

        #In general we use LU solver because it doesn't need the matrix to be symmetric and definite positive
    du = gedim.LUSolver(stiffness + advection + reaction, \
            forcingTerm_g - forcingTerm_v - forcingTerm_der_v, \
            lib)
    
    #Newton step: we add to the old solution the increment found with the Newton method
    u_k = u_k + du

    #Error computation: con il galerkin classico possiamo calcolare l'errore con le funzioni ComputeError, ma se il metodo non è quello classico, non è detto esistano le rispettive funzioni
    du_normL2 = gedim.ComputeErrorL2(Zeros, du, np.zeros(problemData['NumberStrongs'], order='F'), lib)
    u_errorL2 = gedim.ComputeErrorL2(Burger_exactSolution, u_k, u_strong, lib)
    u_errorH1 = gedim.ComputeErrorH1(Burger_exactDerivativeSolution, u_k, u_strong, lib) 
    u_normL2 = gedim.ComputeErrorL2(Zeros, u_k, u_strong, lib) # we aim it is zero, so we pass to it the zero function
    u_normH1 = gedim.ComputeErrorH1(ZerosDerivative, u_k, u_strong, lib) #Similarly t the previous, we pass the function with derivative null along each direction
    
    solution_norm = u_normL2;
    residual_norm = du_normL2;
    
    print("dofs", "h", "errorL2", "errorH1", "residual", "iteration", "max_iteration")
    print(problemData['NumberDOFs'], '{:.16e}'.format(problemData['H']), '{:.16e}'.format(u_errorL2 / u_normL2), '{:.16e}'.format(u_errorH1 / u_normH1), '{:.16e}'.format(residual_norm / u_normL2), '{:d}'.format(num_iteration), '{:d}'.format(max_iterations)) 
    
    num_iteration = num_iteration + 1

    #In order to achieve a good approximation (error under the tolerance) are needed just 4 iteration of the Newton schema

### Plot Solution

In [ ]:
#Visualizzazione della soluzione e calcolo dell'errore FONDAMENTALI NEL PROGETTO !!!
gedim.PlotSolution(mesh, dofs, strongs, u_k, u_strong)

#PLotSolution implements in Pyton the two plot of 2D and 3D plots of the solution. IT's better to export the solution in order to visulize the solution in other applications, as ParaView
#ExportSolution give us a file .inp which is the file format of the UCD structur file. It's limitated to specific type of geometrical elemntsù: tringle hexahedron, thetraedron, pyramidon, line, points.
# Filter "Warp by scalar" in ParaView let choos one of the scalar associated to the solution... to 3D visualization
gedim.ExportSolution(Burger_exactSolution, u_k, u_strong, lib)

#Nella costruzione delle funzioni non ci interessava sapere chi fossero i nodi di quadratura per la costruzione delle matrici. In questo caso invece, dovendo valutare l'errore in tutte
#le celle dobbiamo conoscere i nodi di quadratura in tutto il dominio e li salviamo nella matrice globale quadraturePoints
#Inoltre per ogni nodo di quadratura dobbiamo conoscere il peso per cui dobbiamo moltiplicare, altrimenti l'approssimazione dell'integrale non è corretta. 
#La sommma dei pesi è 1 poichè approssimo l'integrale della funzione unitaria che fornisce la misura del nostro dominio
[numQuadraturePoints, quadraturePoints, quadratureWeights, sol, sol_x, sol_y] = gedim.EvaluateSolutionOnPoints(u_k, u_strong, lib)

#If the order of FE is 2 the PlotSoution does not use the info on the middle points of the FE but just that on the vertices
#In order to taking account also of the dofs that are not equal to the vertices we use ExtractSolutionOnPoints.
#We have a collection of points and in each point we have a value of the solution: we have the galerkin numerical solution evaluated in each qadrature formula. 
#We don't have the solution on the cells.
#In order to have the solution on cells we have to add a filter to the numeric solution that is associated to each point of the mesh, not to the cell. We need to change cell
#array to point array: using cellDataPointData.
#Then, we have to generate a mesh: we use the Delauneay 2D from the filter on the solution on points. We obtain a mesh conformed to our points.
#The borders are missing because the quadrature formula is implemented INSIDE the cells. We add the warpByScalar filter and we see the expected better approximation of the exact solution
gedim.ExportSolutionOnPoints(numQuadraturePoints, quadraturePoints, sol, lib) 


#If we have a network, notice that sol is an array with shape 864, and for each point we have the evaluation of the solution on each point. In Galerkin it is correct, but not in NN
#In the latter case we have to evaluate the network in quadrature points adding the line 
#nn(QuadraturePoints)
#before the export.